In [35]:
import selenium 
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import os
import pandas as pd
from selenium.webdriver.common.keys import Keys

In [36]:
print('-'*100)
print('The U.S. FDA COVID Alerts crawler')
print('='*100)

cnt = int(input('1.몇 건의 코로나 알림 데이터를 수집하시겠습니까? '))
f_dir = input('2.파일 저장 경로를 입력하세요 : ')
if f_dir == "" : 
    f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
os.makedirs(f_dir+s+'-'+'U.S. FDA COVID ALERTS')
os.chdir(f_dir+s+'-'+'U.S. FDA COVID ALERTS')

f_name = s+'-'+'FDA COVID ALERTS.txt'
fc_name = s+'-'+'FDA COVID ALERTS.csv'
fx_name = s+'-'+'FDA COVID ALERTS.xlsx'

----------------------------------------------------------------------------------------------------
The U.S. FDA COVID Alerts crawler
1.몇 건의 코로나 알림 데이터를 수집하시겠습니까? 100
2.파일 저장 경로를 입력하세요 : 


In [37]:
path = '/Users/kibeomkim/Desktop/chromedriver_88/chromedriver'
options= webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')
driver = webdriver.Chrome(path, options=options)
s_time = time.time()
driver.get('https://www.fda.gov/emergency-preparedness-and-response/coronavirus-disease-2019-covid-19/covid-19-vaccines#news')
time.sleep(5)

In [38]:
no = 1
page_no = 1
f = open(f_name, 'a', encoding='utf-8')
year_list1 = []
day_list1 = []
month_list1 = []
title_list1 = []
article_list1 = []

while True : 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table', id='DataTables_Table_0').find('tbody')
    tr_list = table.find_all('tr')
    
    for contents in tr_list : 
        date = contents.find_all('td')[0].get_text().replace('\n', "").strip()
        detail_date = date.split('/')
        year = detail_date[2]
        day = detail_date[1]
        month = detail_date[0]
        
        year_list1.append(year)
        f.write('1.연도 : %d' %int(year)+'\n')
        print('1.연도 : %d' %int(year))
        
        month_list1.append(month)
        f.write('2.월 : %d' %int(month)+'\n')
        print('2.월 : %d' %int(month))
        
        day_list1.append(day)
        f.write('3.일자 : %d' %int(day)+'\n')
        print('3.일자 : %d' %int(day))
        
        title = contents.find_all('td')[1].find('a').get_text().replace('\n', "").strip()
        f.write('4.뉴스 제목 : %s' %title+'\n')
        print('4.뉴스 제목 : %s' %title)
        title_list1.append(title)
        
        try : 
            article = contents.find_all('td')[1].find_all('p')[1].get_text().replace('\n', "").strip()
            f.write('5.뉴스 내용 : %s' %article+'\n')
            print('5.뉴스 내용 : %s' %article)
            article_list1.append(article)
            print()
        except : 
            article_list1.append(None)
            f.write('5.뉴스 내용 : not available'+'\n')
            print('5.뉴스 내용 : not available')
            
        f.write('\n')
        
        no += 1
        if no > cnt : 
            break
    if no > cnt :
        break
    page_no += 1
    try : 
        driver.find_element_by_link_text('%d' %page_no).click()
        time.sleep(5)
    except : 
        break
        
f.close()
driver.quit()

fda = pd.DataFrame()
fda['year'] = year_list1
fda['month'] = month_list1
fda['day'] = day_list1
fda['title'] = title_list1
fda['article'] = article_list1
fda.to_csv(fc_name, encoding='utf-8-sig')
fda.to_excel(fx_name)

e_time = time.time()
total_time = e_time - s_time 

print('-'*100)
print('요청하신 총 %s 건 중 %s 건 만큼을 수집했습니다' %(cnt, no-1))
print('총 소요시간은 %s초 입니다' %round(total_time, 2))
print('='*100)

        


1.연도 : 2021
2.월 : 2
3.일자 : 26
4.뉴스 제목 : Vaccines and Related Biological Products Advisory Committee
5.뉴스 내용 : The Vaccines and Related Biological Products Advisory Committee will meet in open session to discuss Emergency Use Authorization (EUA) of the Janssen Biotech Inc. COVID-19 Vaccine for active immunization to prevent COVID-19 caused by SARS-CoV-2 in individuals 18 years and older.

1.연도 : 2021
2.월 : 2
3.일자 : 23
4.뉴스 제목 : Coronavirus (COVID-19) Update
5.뉴스 내용 : FDA issued guidances for medical product developers to address the emergence and potential future emergence of variants of SARS-CoV-2.

1.연도 : 2021
2.월 : 2
3.일자 : 22
4.뉴스 제목 : Coronavirus (COVID-19) Update: FDA Issues Policies to Guide Medical Product Developers Addressing Virus Variants
5.뉴스 내용 : Today, the FDA issued guidances for medical product developers, specifically covering vaccines, diagnostics and therapeutics products, to address the emergence and potential future emergence of variants of SARS-CoV-2, the virus th

1.연도 : 2020
2.월 : 11
3.일자 : 30
4.뉴스 제목 : Coronavirus (COVID-19) Update: FDA Announces Advisory Committee Meeting to Discuss Second COVID-19 Vaccine Candidate
5.뉴스 내용 : The FDA has scheduled a meeting of its Vaccines and Related Biological Products Advisory Committee (VRBPAC) on Dec. 17 to discuss the request for emergency use authorization (EUA) for a COVID-19 vaccine from Moderna Inc.

1.연도 : 2020
2.월 : 11
3.일자 : 20
4.뉴스 제목 : Coronavirus (COVID-19) Update: FDA Announces Advisory Committee Meeting to Discuss COVID-19 Vaccine Candidate
5.뉴스 내용 : The FDA has scheduled a meeting of its Vaccines and Related Biological Products Advisory Committee (VRBPAC) on Dec. 10 to discuss the request for emergency use authorization (EUA) of a COVID-19 vaccine from Pfizer, Inc. in partnership with BioNTech Manufacturing GmbH.

1.연도 : 2020
2.월 : 11
3.일자 : 20
4.뉴스 제목 : Vaccine Development 101
5.뉴스 내용 : not available
1.연도 : 2020
2.월 : 11
3.일자 : 19
4.뉴스 제목 : FDA and Vaccinate Your Family Talk COVID With Min

1.연도 : 2020
2.월 : 6
3.일자 : 30
4.뉴스 제목 : Coronavirus (COVID-19) Update: FDA Takes Action to Help Facilitate Timely Development of Safe, Effective COVID-19 Vaccines
5.뉴스 내용 : The FDA published a guidance document to facilitate the timely development of safe and effective vaccines to prevent COVID-19, giving recommendations for those developing COVID-19 vaccines for the ultimate purpose of licensure.

1.연도 : 2020
2.월 : 6
3.일자 : 25
4.뉴스 제목 : Partnering with the European Union and Global Regulators on COVID-19
5.뉴스 내용 : The FDA and the European Union, including the European Commission and its European Medicines Agency, are collaborating on many scientific and regulatory fronts as part of our COVID-19 response.

1.연도 : 2020
2.월 : 6
3.일자 : 23
4.뉴스 제목 : Oversight of the Trump Administration’s Response to the COVID-19 Pandemic
5.뉴스 내용 : FDA Commissioner Stephen M. Hahn, M.D., will be testifying for the FDA.Written testimony

1.연도 : 2020
2.월 : 6
3.일자 : 1
4.뉴스 제목 : Remarks by Commissioner Stephen